In [26]:
import pandas as pd

train_df = pd.read_csv('./input/dataset/train.csv')
test_df = pd.read_csv('./input/dataset/test.csv')

del test_df['ID']
del train_df['ID']

In [27]:
# csv 파일을 읽어서 딕셔너리로 변환하는 함수
def csv_to_dict(filename):
    # csv 파일을 데이터프레임으로 읽기
    df = pd.read_csv(filename, encoding='utf-8')
    # 딕셔너리로 변환
    my_dict = df.to_dict(orient='records')
    # before와 after를 키-값으로 매핑한 딕셔너리 생성
    new_dict = {}
    for item in my_dict:
        new_dict[item['before'].strip()] = item['after'].strip()
    return new_dict

# 예시 파일 이름과 경로
filename = './input/word_dict.csv'

# csv 파일을 딕셔너리로 변환
word_dict = csv_to_dict(filename)
word_key_set = set(word_dict.keys())

In [25]:
import re

text = "HelloOOO, how are yoOooou today? I hope everythOoOing is going well."
clean_text = re.sub(r'[Oo]{3,}', '', text)

print(clean_text)

Hell, how are yu today? I hope everything is going well.


In [28]:
import re

def text_preprocessing(text):
    
    # 가장먼저 word_dict 적용
    for key,value in word_dict.items():
        text = text.replace(key,value)
    
    # 이후 문장 맨앞뒤에 있는 . 특수문자 제거",
    text = text.replace(',".()[]~!*-=<>@%$^&','')
    
    # 한국어,공백,숫자 뺴고 모두 삭제 알파벳 O 는 제외함
    text = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣\s\dOo]+", "", text)
    
    # 문장 맨앞뒤 공백 제거
    text = text.strip()
    
    # 문장 맨앞 ㄴ 삭제
    text = re.sub(r"^ㄴ+", "", text)

    # 동일문자 3번이상 반복될시 세번째 나오는 글씨부터 그이상 삭제
    pattern = re.compile(r"([ㄱ-ㅎㅏ-ㅣ가-힣])\1\1+")
    text = pattern.sub(lambda x: x.group(1)*2, text)
    
    # 마지막으로 word_dict 한번더적용
    for key,value in word_dict.items():
        text = text.replace(key,value)
    
    # 5.18은 냅두고 18 은 -> 시발로 변경
    if ("5.18" not in text) and ("18" in text):
        text = text.replace("18",'시발') 
    
    # OoOo 처럼 예외적인경우 다없애기
    text = re.sub(r'[Oo]{3,}', '', text)
    
    return text

In [29]:
from konlpy.tag import Okt  
okt=Okt()  

test_df['processed_text'] = test_df['text'].map(text_preprocessing)
train_df['processed_text'] = train_df['text'].map(text_preprocessing)

train_df['tokenized_text'] = train_df['processed_text'].apply(lambda x: okt.morphs(x))
test_df['tokenized_text'] = test_df['processed_text'].apply(lambda x: okt.morphs(x))

In [30]:
train_speech_text = " ".join(train_df.processed_text.to_numpy().tolist())
test_speech_text = " ".join(test_df.processed_text.to_numpy().tolist())

all_speech_text = train_speech_text + test_speech_text
with open('./middle/all_speech_text.txt', 'w', encoding='utf-8') as file:
    file.write(all_speech_text)

In [31]:
from collections import Counter

with open('./middle/all_speech_text.txt', 'r', encoding='utf-8') as f:
    text = f.read()

all_morphs = okt.morphs(text)
counted_morphs = Counter(all_morphs)

In [32]:
# Dictionary로 변환
top_2000_tokens = counted_morphs.most_common(2000)
top_2000_dict = {}
for word, count in top_2000_tokens:
    top_2000_dict[word] = count

# DataFrame으로 변환하여 출력
top_2000_df = pd.DataFrame.from_dict(top_2000_dict, orient='index', columns=['count'])
top_2000_df.to_excel("./middle/token_counter.xlsx")

In [33]:
# 토큰에서 
# 들, 이, 가 ,은, 에 , 도 ,는 , 을 , 다, 아 , 의 , 한 , 를 ,1,2,3,4,5,6,7,8,9 제거

# 최소한 제거 
stop_words = ["들","이","가","은","에","도","는","을","다","아",
              "의","한","를","1","2","3","4","5","6","7",
              "8","9"]

print( f"before 갯수 : {len(counted_morphs)}")
subtracted_counted_morphs = Counter(
    {word: count for word, count in counted_morphs.items() 
     if word not in stop_words}
)

print(f"after 갯수 : {len(subtracted_counted_morphs)}")

before 갯수 : 80096
after 갯수 : 80074


In [34]:
def remove_stopwords(text):
    stop_words = ["들","이","가","은","에","도","는","을","다","아",
              "의","한","를","1","2","3","4","5","6","7",
              "8","9"]
    stop_word_set = set(stop_words)
    ret_list = []
    for tmp_token in text:
        if tmp_token in stop_word_set:
            continue
        else:
            ret_list.append(tmp_token)
    
    return ret_list

# stop_words 제거
test_df['tokenized_text'] = test_df['tokenized_text'].map(remove_stopwords)
train_df['tokenized_text'] = train_df['tokenized_text'].map(remove_stopwords)
token_counter_df = pd.DataFrame(subtracted_counted_morphs.items(),columns=["단어","빈도수"])

In [35]:
test_df.to_excel('./middle/after_tokenized_test.xlsx',encoding='utf-8', index=False)
train_df.to_excel('./middle/after_tokenized_train.xlsx',encoding='utf-8', index=False)
token_counter_df.to_excel('./middle/all_tokens.xlsx',encoding='utf-8',index=False)

In [36]:
a = "한남충들 미투에 얼마나 관심을 가졌다고 지겹다고 웅앵웅이노 앙 재기민기띠ㅋㅋ"

a = text_preprocessing(a)
print(a)

okt.morphs(a)

한남충들 미투에 얼마나 관심을 가졌다고 지겹다고 웅앵웅이노 앙 재기민기띠ㅋㅋ


['한남충',
 '들',
 '미투',
 '에',
 '얼마나',
 '관심',
 '을',
 '가졌다고',
 '지겹다고',
 '웅앵웅',
 '이노',
 '앙',
 '재기',
 '민',
 '기',
 '띠',
 'ㅋㅋ']